<a href="https://colab.research.google.com/github/haitam-zouhair/workspace_henceforth/blob/master/french_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%%shell
cp -r 'drive/My Drive/Projects/Zouhair/' .

In [3]:
import os
os.chdir("Zouhair")

In [4]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 13.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=e9d122e99e035a024ad79771a930ada6fa8f047dc67906a31aa5af5aaec3f347
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=a4cb76daecc6be247958167e7e1982116a9ae8c7239d99f6dd52e236073237b6
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


# Dataset

I used [webhose.io](https://webhose.io/) as dataset. Dataset can be downloaded [here](https://webhose.io/free-datasets/french-news-articles/), you can also use directly the dataset from Google Drive, I put them [here](https://drive.google.com/file/d/14KLq_RGJokiqbin91hOBp7-OOLdtbieE/view?usp=sharing).

In [5]:
%%shell
unzip french.zip
mkdir data/
mv 636_webhose-2016-10_20170904085710.zip data/
cd data/
unzip 636_webhose-2016-10_20170904085710.zip
rm 636_webhose-2016-10_20170904085710.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: news_0240310.json       
  inflating: news_0240311.json       
  inflating: news_0240312.json       
  inflating: news_0240313.json       
  inflating: news_0240314.json       
  inflating: news_0240315.json       
  inflating: news_0240316.json       
  inflating: news_0240317.json       
  inflating: news_0240318.json       
  inflating: news_0240319.json       
  inflating: news_0240320.json       
  inflating: news_0240321.json       
  inflating: news_0240322.json       
  inflating: news_0240323.json       
  inflating: news_0240324.json       
  inflating: news_0240325.json       
  inflating: news_0240326.json       
  inflating: news_0240327.json       
  inflating: news_0240328.json       
  inflating: news_0240329.json       
  inflating: news_0240330.json       
  inflating: news_0240331.json       
  inflating: news_0240332.json       
  inflating: news_0240333.json       
  inflatin

# I/-French Text Processing

Similar to English text, french text should be preporcessed before performing any NLP task on it. The pre-processing steps are roughly the same:

* **Tokenization:**
* **Remove Stopwords:** 
* **Lemmatization:**
* **Stemming:**



In [6]:
import os
import json
from tqdm.notebook import tqdm

from gensim.utils import simple_preprocess
from nltk.stem.snowball import FrenchStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


def get_stopswords():
    """
    get French stopwords from the nltk kit
    """
    raw_stopword_list = raw_stopword_list = ["Ap.", "Apr.", "GHz", "MHz", "USD", "a", "afin", "ah", "ai", "aie", "aient", "aies", "ait", "alors", "après", "as", "attendu", "au", "au-delà", "au-devant", "aucun", "aucune", "audit", "auprès", "auquel", "aura", "aurai", "auraient", "aurais", "aurait", "auras", "aurez", "auriez", "aurions", "aurons", "auront", "aussi", "autour", "autre", "autres", "autrui", "aux", "auxdites", "auxdits", "auxquelles", "auxquels", "avaient", "avais", "avait", "avant", "avec", "avez", "aviez", "avions", "avons", "ayant", "ayez", "ayons", "b", "bah", "banco", "ben", "bien", "bé", "c", "c'", "c'est", "c'était", "car", "ce", "ceci", "cela", "celle", "celle-ci", "celle-là", "celles", "celles-ci", "celles-là", "celui", "celui-ci", "celui-là", "celà", "cent", "cents", "cependant", "certain", "certaine", "certaines", "certains", "ces", "cet", "cette", "ceux", "ceux-ci", "ceux-là", "cf.", "cg", "cgr", "chacun", "chacune", "chaque", "chez", "ci", "cinq", "cinquante", "cinquante-cinq", "cinquante-deux", "cinquante-et-un", "cinquante-huit", "cinquante-neuf", "cinquante-quatre", "cinquante-sept", "cinquante-six", "cinquante-trois", "cl", "cm", "cm²", "comme", "contre", "d", "d'", "d'après", "d'un", "d'une", "dans", "de", "depuis", "derrière", "des", "desdites", "desdits", "desquelles", "desquels", "deux", "devant", "devers", "dg", "différentes", "différents", "divers", "diverses", "dix", "dix-huit", "dix-neuf", "dix-sept", "dl", "dm", "donc", "dont", "douze", "du", "dudit", "duquel", "durant", "dès", "déjà", "e", "eh", "elle", "elles", "en", "en-dehors", "encore", "enfin", "entre", "envers", "es", "est", "et", "eu", "eue", "eues", "euh", "eurent", "eus", "eusse", "eussent", "eusses", "eussiez", "eussions", "eut", "eux", "eûmes", "eût", "eûtes", "f", "fait", "fi", "flac", "fors", "furent", "fus", "fusse", "fussent", "fusses", "fussiez", "fussions", "fut", "fûmes", "fût", "fûtes", "g", "gr", "h", "ha", "han", "hein", "hem", "heu", "hg", "hl", "hm", "hm³", "holà", "hop", "hormis", "hors", "huit", "hum", "hé", "i", "ici", "il", "ils", "j", "j'", "j'ai", "j'avais", "j'étais", "jamais", "je", "jusqu'", "jusqu'au", "jusqu'aux", "jusqu'à", "jusque", "k", "kg", "km", "km²", "l", "l'", "l'autre", "l'on", "l'un", "l'une", "la", "laquelle", "le", "lequel", "les", "lesquelles", "lesquels", "leur", "leurs", "lez", "lors", "lorsqu'", "lorsque", "lui", "lès", "m", "m'", "ma", "maint", "mainte", "maintes", "maints", "mais", "malgré", "me", "mes", "mg", "mgr", "mil", "mille", "milliards", "millions", "ml", "mm", "mm²", "moi", "moins", "mon", "moyennant", "mt", "m²", "m³", "même", "mêmes", "n", "n'avait", "n'y", "ne", "neuf", "ni", "non", "nonante", "nonobstant", "nos", "notre", "nous", "nul", "nulle", "nº", "néanmoins", "o", "octante", "oh", "on", "ont", "onze", "or", "ou", "outre", "où", "p", "par", "par-delà", "parbleu", "parce", "parmi", "pas", "passé", "pendant", "personne", "peu", "plus", "plus_d'un", "plus_d'une", "plusieurs", "pour", "pourquoi", "pourtant", "pourvu", "près", "puisqu'", "puisque", "q", "qu", "qu'", "qu'elle", "qu'elles", "qu'il", "qu'ils", "qu'on", "quand", "quant", "quarante", "quarante-cinq", "quarante-deux", "quarante-et-un", "quarante-huit", "quarante-neuf", "quarante-quatre", "quarante-sept", "quarante-six", "quarante-trois", "quatorze", "quatre", "quatre-vingt", "quatre-vingt-cinq", "quatre-vingt-deux", "quatre-vingt-dix", "quatre-vingt-dix-huit", "quatre-vingt-dix-neuf", "quatre-vingt-dix-sept", "quatre-vingt-douze", "quatre-vingt-huit", "quatre-vingt-neuf", "quatre-vingt-onze", "quatre-vingt-quatorze", "quatre-vingt-quatre", "quatre-vingt-quinze", "quatre-vingt-seize", "quatre-vingt-sept", "quatre-vingt-six", "quatre-vingt-treize", "quatre-vingt-trois", "quatre-vingt-un", "quatre-vingt-une", "quatre-vingts", "que", "quel", "quelle", "quelles", "quelqu'", "quelqu'un", "quelqu'une", "quelque", "quelques", "quelques-unes", "quelques-uns", "quels", "qui", "quiconque", "quinze", "quoi", "quoiqu'", "quoique", "r", "revoici", "revoilà", "rien", "s", "s'", "sa", "sans", "sauf", "se", "seize", "selon", "sept", "septante", "sera", "serai", "seraient", "serais", "serait", "seras", "serez", "seriez", "serions", "serons", "seront", "ses", "si", "sinon", "six", "soi", "soient", "sois", "soit", "soixante", "soixante-cinq", "soixante-deux", "soixante-dix", "soixante-dix-huit", "soixante-dix-neuf", "soixante-dix-sept", "soixante-douze", "soixante-et-onze", "soixante-et-un", "soixante-et-une", "soixante-huit", "soixante-neuf", "soixante-quatorze", "soixante-quatre", "soixante-quinze", "soixante-seize", "soixante-sept", "soixante-six", "soixante-treize", "soixante-trois", "sommes", "son", "sont", "sous", "soyez", "soyons", "suis", "suite", "sur", "sus", "t", "t'", "ta", "tacatac", "tandis", "te", "tel", "telle", "telles", "tels", "tes", "toi", "ton", "toujours", "tous", "tout", "toute", "toutefois", "toutes", "treize", "trente", "trente-cinq", "trente-deux", "trente-et-un", "trente-huit", "trente-neuf", "trente-quatre", "trente-sept", "trente-six", "trente-trois", "trois", "très", "tu", "u", "un", "une", "unes", "uns", "v", "vers", "via", "vingt", "vingt-cinq", "vingt-deux", "vingt-huit", "vingt-neuf", "vingt-quatre", "vingt-sept", "vingt-six", "vingt-trois", "vis-à-vis", "voici", "voilà", "vos", "votre", "vous", "w", "x", "y", "z", "zéro", "à", "ç'", "ça", "ès", "étaient", "étais", "était", "étant", "étiez", "étions", "été", "étée", "étées", "étés", "êtes", "être", "ô"]
    return raw_stopword_list


def preprocess(text):
    result = []
    stopwords = get_stopswords()
    stemmer = FrenchStemmer()
    for token in simple_preprocess(text):
        if token not in stopwords and len(token) > 3:
            result.append(stemmer.stem(token))
    return result

In [7]:
root_path = "data"
files = os.listdir(root_path)


corpus = []
for file_name in tqdm(files):
    file_path = os.path.join(root_path, file_name)
    with open(file_path, "r") as f:
        text = json.load(f)["text"]
        corpus.append(preprocess(text))


# II/- Topic modeling using LDA 


In [9]:
from gensim import corpora, models

import numpy as np

## 1. Bag of Words

In [10]:
# build bag of words dictionnary
dictionary = corpora.Dictionary(corpus)

# filter iinfrequent words
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]

View Bag of Word result

In [11]:
for i in range(len(bow_corpus[0])):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus[0][i][0], 
                                               dictionary[bow_corpus[0][i][0]], 
                                               bow_corpus[0][i][1]))

Word 0 ("acheampong") appears 1 time.
Word 1 ("acquis") appears 1 time.
Word 2 ("actuel") appears 1 time.
Word 3 ("anderlecht") appears 6 time.
Word 4 ("averag") appears 1 time.
Word 5 ("badj") appears 1 time.
Word 6 ("banc") appears 2 time.
Word 7 ("battu") appears 1 time.
Word 8 ("boeckx") appears 1 time.
Word 9 ("bolingol") appears 1 time.
Word 10 ("breydel") appears 1 time.
Word 11 ("brug") appears 4 time.
Word 12 ("bruno") appears 1 time.
Word 13 ("but") appears 1 time.
Word 14 ("butel") appears 1 time.
Word 15 ("capel") appears 1 time.
Word 16 ("championnat") appears 1 time.
Word 17 ("classiqu") appears 1 time.
Word 18 ("claudem") appears 1 time.
Word 19 ("club") appears 1 time.
Word 20 ("comment") appears 2 time.
Word 21 ("compos") appears 1 time.
Word 22 ("compétit") appears 1 time.
Word 23 ("conced") appears 1 time.
Word 24 ("cool") appears 1 time.
Word 25 ("dat") appears 1 time.
Word 26 ("dendonck") appears 1 time.
Word 27 ("denswil") appears 1 time.
Word 28 ("derni") appears

## 2. Running LDA

In [12]:
NUM_TOPICS = 6

lda_model = models.LdaMulticore(bow_corpus,
                                num_topics=NUM_TOPICS,
                                id2word=dictionary,
                                passes=2,
                                workers=4)

In [13]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} | Word: {}'.format(idx, topic))

Topic: 0 | Word: 0.007*"premi" + 0.006*"match" + 0.006*"derni" + 0.006*"franc" + 0.005*"équip" + 0.005*"saison" + 0.004*"trump" + 0.004*"point" + 0.004*"joueur" + 0.004*"octobr"
Topic: 1 | Word: 0.053*"pouv" + 0.053*"don" + 0.050*"indiqu" + 0.037*"annonc" + 0.036*"raison" + 0.035*"merc" + 0.034*"insult" + 0.034*"original" + 0.024*"veuill" + 0.022*"mail"
Topic: 2 | Word: 0.009*"minut" + 0.005*"euros" + 0.005*"franc" + 0.005*"heur" + 0.004*"group" + 0.004*"premi" + 0.004*"européen" + 0.004*"entrepris" + 0.004*"march" + 0.004*"anné"
Topic: 3 | Word: 0.005*"franc" + 0.005*"octobr" + 0.005*"président" + 0.005*"part" + 0.005*"droit" + 0.004*"ministr" + 0.004*"person" + 0.004*"polit" + 0.004*"etat" + 0.004*"premi"
Topic: 4 | Word: 0.019*"utilis" + 0.013*"typ" + 0.011*"annonc" + 0.010*"descript" + 0.010*"envoi" + 0.010*"russ" + 0.008*"offre" + 0.007*"mail" + 0.007*"compt" + 0.007*"insert"
Topic: 5 | Word: 0.004*"grand" + 0.004*"fair" + 0.004*"jour" + 0.003*"enfant" + 0.003*"femm" + 0.003*"octo

# III/- Evaluation

## 1) Vizualisation

In [14]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.152599 -0.031299       1        1  24.018522
2     -0.132106  0.028280       2        1  23.576952
5     -0.121177  0.006870       3        1  21.046686
0     -0.164324 -0.098910       4        1  18.620239
1      0.435460 -0.097334       5        1   7.303563
4      0.134746  0.192394       6        1   5.434040, topic_info=         Term           Freq          Total Category  logprob  loglift
107       don  178494.000000  178494.000000  Default  30.0000  30.0000
144      pouv  142705.000000  142705.000000  Default  29.0000  29.0000
120    indiqu  152555.000000  152555.000000  Default  28.0000  28.0000
89     annonc  164091.000000  164091.000000  Default  27.0000  27.0000
148    raison  109200.000000  109200.000000  Default  26.0000  26.0000
..        ...            ...            ...      ...      ...      ...
114     grand    9715.681641   96792.421875   Topic6  -5.2325   0.6137
113     fraud    7886.043457   47691.281250   Topic6  -5.4411   1.1128
124  intéress    7674.562988   59874.667969   Topic6  -5.4683   0.8581
129    messag    6632.926758   27388.402344   Topic6  -5.6142   1.4944
116      grâc    6662.086914   40098.074219   Topic6  -5.6098   1.1176

[599 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
32704      5  0.963046  accommazzo
1620       1  0.282139      accord
1620       2  0.557858      accord
1620       3  0.018668      accord
1620       4  0.088248      accord
...      ...       ...         ...
5239       2  0.013723       évacu
5239       3  0.009389       évacu
5239       4  0.011195       évacu
5239       5  0.000241       évacu
5239       6  0.006982       évacu

[2356 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 6, 1, 2, 5])

## 2. Perplexity and Coherence Score

In [16]:
coherence_model_lda = models.CoherenceModel(model=lda_model,
                                     texts=corpus,
                                     dictionary=dictionary,
                                     coherence='c_v')

print('Coherence Score: {:.2f}'.format(coherence_model_lda.get_coherence()))

Coherence Score: 0.49


# IV/- Topic based text retreival

In this section, we use the results of topic modeling in order to build a content classifier for text retreival. Each document will be represented by a `NUM_TOPICS`-vector capturing the distribution of the learnt topics. This is similar to the approach used in [Phan et al. (2008)](http://gibbslda.sourceforge.net/fp224-phan.pdf)

## 1. Build dataset

In [17]:
N = len(bow_corpus)


X = np.zeros((N, NUM_TOPICS))





for i in range(N):
    topics_proba = lda_model.get_document_topics(bow_corpus[i],
                                              minimum_probability=0.0)
    
    topic_probas = [v for _, v in  topics_proba] 
    X[i,:] = np.array(topic_probas)
    
    

## 2. Train unsupervised classifier

We train some simple classifiers using the extracted feature vectors

In [19]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=20)
neigh.fit(X)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

# V/- Model Deployment

In this section we save the model, and we load it into `lda_model_uploaded` and test it on new text.



In [20]:
import warnings
warnings.filterwarnings('ignore')


model_path = "model"

lda_model.save(model_path)
lda_model_loaded = lda_model.load(model_path)

In [23]:
! cp "/content/Zouhair/model" "/content/drive/My Drive/Projects/Zouhair/French_model/"

In [24]:
! cp "/content/Zouhair/model.expElogbeta.npy" "/content/drive/My Drive/Projects/Zouhair/French_model/"

In [25]:
! cp "/content/Zouhair/model.id2word" "/content/drive/My Drive/Projects/Zouhair/French_model/"

In [26]:
! cp "/content/Zouhair/model.state" "/content/drive/My Drive/Projects/Zouhair/French_model/"

In [28]:
#get a random text
text = " Début 2011, le printemps arabe atteint la Syrie. Des manifestations en faveur de la démocratie et majoritairement pacifiques ont lieu dans tout le pays contre le régime baasiste du président Bachar el-Assad19. Ces manifestations sont réprimées brutalement par le régime et peu à peu, le mouvement de contestation se transforme en une rébellion armée. De nombreux belligérants participent au conflit. L'Armée syrienne libre, nationaliste, est le premier mouvement à mener la rébellion mais à partir de 2013, elle est supplantée par des groupes islamistes sunnites comme Ahrar al-Cham, Jaych al-Islam ou le Front al-Nosra, la branche syrienne d'al-Qaïda. Les groupes rebelles sont soutenus principalement par la Turquie, le Qatar et l'Arabie saoudite. De son côté, le régime syrien se maintient grâce à l'aide de l'Iran et de la Russie. L'Iran déploie en Syrie des forces du Corps des Gardiens de la révolution islamique dès le début du conflit, par la suite des dizaines de milices islamistes chiites parrainées par Téhéran gagnent à leur tour la Syrie ; comme les Libanais du Hezbollah, les Afghans hazaras de la Division des Fatimides ou les Irakiens de l'Organisation Badr et du Harakat Hezbollah al-Nujaba. De son côté la Russie intervient militairement en septembre 2015 et commence une campagne de frappes aériennes en soutien du régime "

print(text)

# preprocess text
processed_text = preprocess(text)

# get Bag of Words
bow_doc = lda_model_loaded.id2word.doc2bow(processed_text)

# get topics 
topics = lda_model_loaded.get_document_topics(bow_doc,
                                              minimum_probability=0.0)
topic = [v for _, v in  topics] 
query= np.array(topic)

print(neigh.kneighbors([query], return_distance= False))

print("*"*30)

print([topics])

 Début 2011, le printemps arabe atteint la Syrie. Des manifestations en faveur de la démocratie et majoritairement pacifiques ont lieu dans tout le pays contre le régime baasiste du président Bachar el-Assad19. Ces manifestations sont réprimées brutalement par le régime et peu à peu, le mouvement de contestation se transforme en une rébellion armée. De nombreux belligérants participent au conflit. L'Armée syrienne libre, nationaliste, est le premier mouvement à mener la rébellion mais à partir de 2013, elle est supplantée par des groupes islamistes sunnites comme Ahrar al-Cham, Jaych al-Islam ou le Front al-Nosra, la branche syrienne d'al-Qaïda. Les groupes rebelles sont soutenus principalement par la Turquie, le Qatar et l'Arabie saoudite. De son côté, le régime syrien se maintient grâce à l'aide de l'Iran et de la Russie. L'Iran déploie en Syrie des forces du Corps des Gardiens de la révolution islamique dès le début du conflit, par la suite des dizaines de milices islamistes chiites